In [1]:
# Below is a copy of slurm_train.py from anurag/slurm_onebutton

import torchvision

# import some common libraries
import numpy as np
import sys
if '/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

import cv2
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import random
import os
import numpy as np
import json

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.config import CfgNode as CN
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data import DatasetMapper, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import detectron2.data.transforms as T
import shutil
from setuptools.namespaces import flatten

import random
import torch 
import base64
import io
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from ast import literal_eval
from PIL import Image


coco_yaml = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
lvis_yaml = "LVIS-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"
lvis_yaml2 = "LVIS-InstanceSegmentation/mask_rcnn_R_101_FPN_1x.yaml"
pano_yaml = "COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml"

jsons_root = '/checkpoint/apratik/finals/jsons/active_vision/'
img_dir_test = '/checkpoint/apratik/ActiveVision/active_vision/replica_random_exploration_data/frl_apartment_1/rgb'
test_jsons = ['frlapt1_20n0.json', 'frlapt1_20n1.json', 'frlapt1_20n2.json']
test_jsons = [os.path.join(jsons_root, x) for x in test_jsons]

# val_json = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200/83/seg/coco_train.json'
val_json = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_valgtfix.json'
# img_dir_val = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200/83/rgb'
img_dir_val = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb'

## Detectron2 Setup

# from copy_paste import CopyPaste
# import albumentations as A

class Trainer(DefaultTrainer):
#     @classmethod
#     def build_evaluator(cls, cfg, dataset_name, output_folder=None):
#         if output_folder is None:
#             output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
#         return COCOEvaluator(dataset_name, output_dir=output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        mapper = DatasetMapper(cfg, is_train=True, augmentations=[
            T.ResizeShortestEdge(short_edge_length=cfg.INPUT.MIN_SIZE_TRAIN, max_size=1333, sample_style='choice'),
            T.RandomFlip(prob=0.5),
            T.RandomCrop("absolute", (640, 640)),
            T.RandomBrightness(0.9, 1.1)
        ])
        return build_detection_train_loader(cfg, mapper=mapper)

class COCOTrain:
    def __init__(self, lr, w, maxiters, seed):
        self.cfg = get_cfg()
        self.cfg.merge_from_file(model_zoo.get_config_file(coco_yaml))
        self.cfg.SOLVER.BASE_LR = lr  # pick a good LR
        self.cfg.SOLVER.MAX_ITER = maxiters
        self.cfg.SOLVER.WARMUP_ITERS = w
        self.seed = seed
        
    def reset(self, train_json, img_dir_train, dataset_name):
        DatasetCatalog.clear()
        MetadataCatalog.clear()
        self.train_data = dataset_name +  "_train"
        self.dataset_name = dataset_name
        self.train_json = train_json
        register_coco_instances(self.train_data, {}, train_json, img_dir_train)
        self.results = {
            "bbox": {
                "AP50": []
            },
            "segm": {
                "AP50": []
            }
        }
        self.val_results = {
            "bbox": {
                "AP50": []
            },
            "segm": {
                "AP50": []
            }
        }
    
    def vis(self):
        dataset_dicts = DatasetCatalog.get(self.train_data)
        for d in random.sample(dataset_dicts, 2):
            img = cv2.imread(d["file_name"])
            visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get(self.train_data), scale=0.5)
            vis = visualizer.draw_dataset_dict(d)
            img = vis.get_image()
            plt.figure(figsize=(12,8))
            plt.imshow(img)
            plt.show()
            
    def train(self):
        cfg = self.cfg
        print(f'SOLVER PARAMS {cfg.SOLVER.MAX_ITER, cfg.SOLVER.WARMUP_ITERS, cfg.SOLVER.BASE_LR}')
        cfg.DATASETS.TRAIN = (self.train_data,)
        cfg.DATASETS.TEST = ()
        cfg.DATALOADER.NUM_WORKERS = 2
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(coco_yaml)  # Let training initialize from model zoo
        cfg.SOLVER.IMS_PER_BATCH = 2
        cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 
        MetadataCatalog.get(self.train_data).thing_classes = ['chair', 'cushion', 'door', 'indoor-plant', 'sofa', 'table']
        print(f'classes {MetadataCatalog.get(self.train_data)}')
        cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(MetadataCatalog.get(self.train_data).get("thing_classes"))  
        cfg.OUTPUT_DIR = os.path.join('output_aug', str(cfg.SOLVER.MAX_ITER), self.dataset_name + str(self.seed))
        print(f"recreating {cfg.OUTPUT_DIR}")
        # if os.path.isdir(cfg.OUTPUT_DIR):
        #     shutil.rmtree(cfg.OUTPUT_DIR)
        print(cfg.OUTPUT_DIR)
        os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
        self.trainer = Trainer(cfg) #DefaultTrainer(cfg) 
        self.trainer.resume_or_load(resume=False)
        self.trainer.train()
        
    def run_val(self, dataset_name, val_json, img_dir_val):
        self.val_data = dataset_name + "_val" + str(self.seed)
        self.val_json = val_json
        self.cfg.DATASETS.TEST = (self.val_data,)
        register_coco_instances(self.val_data, {}, val_json, img_dir_val)
        MetadataCatalog.get(self.val_data).thing_classes = ['chair', 'cushion', 'door', 'indoor-plant', 'sofa', 'table']
        print(f'classes {MetadataCatalog.get(self.val_data)}')
        self.evaluator = COCOEvaluator(self.val_data, ("bbox", "segm"), False, output_dir=self.cfg.OUTPUT_DIR)
        self.val_loader = build_detection_test_loader(self.cfg, self.val_data)
        results = inference_on_dataset(self.trainer.model, self.val_loader, self.evaluator)
        self.val_results['bbox']['AP50'].append(results['bbox']['AP50'])
        self.val_results['segm']['AP50'].append(results['segm']['AP50'])
        return results

    def run_test(self, dataset_name, test_json, img_dir_test):
        self.test_data = dataset_name + "_test" + str(self.seed)
        self.test_json = test_json
        self.cfg.DATASETS.TEST = (self.test_data,)
        register_coco_instances(self.test_data, {}, test_json, img_dir_test)
        MetadataCatalog.get(self.test_data).thing_classes = ['chair', 'cushion', 'door', 'indoor-plant', 'sofa', 'table']
        print(f'classes {MetadataCatalog.get(self.test_data)}')
        self.evaluator = COCOEvaluator(self.test_data, ("bbox", "segm"), False, output_dir=self.cfg.OUTPUT_DIR)
        self.val_loader = build_detection_test_loader(self.cfg, self.test_data)
        results = inference_on_dataset(self.trainer.model, self.val_loader, self.evaluator)
        self.results['bbox']['AP50'].append(results['bbox']['AP50'])
        self.results['segm']['AP50'].append(results['segm']['AP50'])
        return results
        
    def run_train(self, train_json, img_dir_train, dataset_name):
        self.reset(train_json, img_dir_train, dataset_name)
        # self.vis()
        self.train()


# maxiters = [500, 800]
# lrs = [0.0001, 0.0005, 0.001, 0.002, 0.005]
# warmups = [100, 200]
lrs = [0.001, 0.0001]
maxiters = [500, 800]
warmups = [200]

def write_summary_to_file(filename, results, header_str):
    if isinstance(results['bbox']['AP50'][0], list):
        results['bbox']['AP50'] = list(flatten(results['bbox']['AP50']))
        results['segm']['AP50'] = list(flatten(results['segm']['AP50']))
    with open(filename, "a") as f:
        f.write(header_str)
        f.write(f"\nbbox AP50 {sum(results['bbox']['AP50'])/len(results['bbox']['AP50'])}")
        f.write(f"\nsegm AP50 {sum(results['segm']['AP50'])/len(results['segm']['AP50'])}")
        f.write(f'\nall results {results}')

from pathlib import Path
import string

def run_training(out_dir, img_dir_train, n, traj, x, gt, p, tr_json):
    results_dir = os.path.join('results1021_sanity2', str(traj), x, str(gt), str(p))
    if not os.path.isdir(results_dir):
        os.makedirs(results_dir)
    train_json = os.path.join(out_dir, tr_json)
    for lr in lrs:
        for warmup in warmups:
            for maxiter in maxiters:
                results = {
                    "bbox": {
                        "AP50": []
                    },
                    "segm": {
                        "AP50": []
                    }
                }
                for i in range(n):
                    c = COCOTrain(lr, warmup, maxiter, i)
                    dataset_name = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(7))
                    print(f'dataset_name {dataset_name}')
                    c.run_train(train_json, img_dir_train, dataset_name)
                    res_eval = c.run_val(dataset_name, val_json, img_dir_val)
                    with open(os.path.join(results_dir, "validation_results.txt"), "a") as f:
                        f.write(f'lr {lr} warmup {warmup} maxiter {maxiter}\n')
                        f.write(json.dumps(res_eval) + '\n')
                    for yix in range(len(test_jsons)):
                        r = c.run_test(dataset_name + str(yix), test_jsons[yix], img_dir_test)
                        with open(os.path.join(results_dir, "all_results.txt"), "a") as f:
                            f.write(json.dumps(r) + '\n')
                    print(f'all results {c.results}')
                    results['bbox']['AP50'].append(c.results['bbox']['AP50'])
                    results['segm']['AP50'].append(c.results['segm']['AP50'])
                    write_summary_to_file(os.path.join(results_dir, str(n) + '_granular.txt'), c.results, f'\ntrain_json {train_json}')
                
                itername = str(lr) + ' ' + str(warmup) + ' ' + str(maxiter) + ' ' + str(n)
                write_summary_to_file(os.path.join(results_dir, itername + '_results_averaged.txt'), results, f'\ntrain_json {train_json}, average over {n} runs')
   

In [9]:
data_path = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019'
# job_folder = '/checkpoint/apratik/jobs/active_vision/pipeline/apartment_0/straightline/no_noise/14-10-2021/19:17:17'
job_folder = '/checkpoint/apratik/jobs/active_vision/pipeline/apartment_0/straightline/no_noise/20-10-2021/23:04:37'
import glob

def _runner(trajs, gt, ps):
    for traj in trajs:
        for p in ps:
            for x in ['default']:
                traj_path = os.path.join(data_path, str(traj), x)
                if os.path.isdir(traj_path):
                    outdir = os.path.join(job_folder, str(traj), x, f'pred_label_gt{gt}p{p}')
                    if len(glob.glob1(outdir,"*.npy")) > 0:
                        run_training(outdir, os.path.join(traj_path, 'rgb'), 1, traj, x, gt, p)
    
            

In [7]:
# have a bunch of json
# /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_gt_0.json
# '/checkpoint/apratik/jobs/active_vision/pipeline/apartment_0/straightline/no_noise/20-10-2021/23:04:37/1/default/seg/coco_gt_0.json'

train_jsons = [f'coco_gt_{p}.json' for p in [0,2,4,6,8]]

data_path = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019'
job_folder = '/checkpoint/apratik/jobs/active_vision/pipeline/apartment_0/straightline/no_noise/20-10-2021/23:04:37'

gt = 5
traj = '1'
x = 'default'
traj_path = os.path.join(data_path, str(traj), x)

train_json = f'coco_gt_0.json'
out_dir = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/'
# os.path.join(job_folder, traj, x, 'seg')
# out_dir = os.path.join(job_folder, traj, x, 'seg')
run_training(out_dir, os.path.join(traj_path, 'rgb'), 1, str(traj), x, gt, 0, train_json)


# for p in [2,4,6,8]:
#     train_json = f'coco_gt_{p}.json'
#     out_dir = os.path.join(job_folder, traj, x, f'pred_label_gt{gt}p{p}')
#     run_training(out_dir, os.path.join(traj_path, 'rgb'), 1, traj, x, gt, p, train_json)
    


dataset_name 060J9CV
SOLVER PARAMS (500, 200, 0.001)
classes Metadata(evaluator_type='coco', image_root='/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb', json_file='/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/seg/coco_gt_0.json', name='060J9CV_train', thing_classes=['chair', 'cushion', 'door', 'indoor-plant', 'sofa', 'table'])
recreating output_aug/500/060J9CV0
output_aug/500/060J9CV0
[10/25 13:48:17 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_s

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[10/25 13:48:18 d2.engine.train_loop]: Starting training from iteration 0


/private/home/apratik/.conda/envs/locobot_env/lib/python3.7/site-packages/detectron2/structures/masks.py:348: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  item = item.nonzero().squeeze(1).cpu().numpy().tolist()
/private/home/apratik/.conda/envs/locobot_env/lib/python3.7/site-packages/detectron2/structures/masks.py:348: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  item = item.nonzero().squeeze(1).cpu().numpy().tolist()
/private/home/apratik/.conda/envs/locobot_env/lib/python3.7/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:103: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of th

[10/25 13:48:22 d2.utils.events]:  eta: 0:01:23  iter: 19  total_loss: 3.319  loss_cls: 1.832  loss_box_reg: 0.6239  loss_mask: 0.689  loss_rpn_cls: 0.08345  loss_rpn_loc: 0.02084  time: 0.1741  data_time: 0.0159  lr: 9.5905e-05  max_mem: 1609M
[10/25 13:48:25 d2.utils.events]:  eta: 0:01:20  iter: 39  total_loss: 2.072  loss_cls: 0.6481  loss_box_reg: 0.6618  loss_mask: 0.6692  loss_rpn_cls: 0.02597  loss_rpn_loc: 0.02788  time: 0.1743  data_time: 0.0029  lr: 0.0001958  max_mem: 1609M
[10/25 13:48:29 d2.utils.events]:  eta: 0:01:17  iter: 59  total_loss: 1.954  loss_cls: 0.5849  loss_box_reg: 0.774  loss_mask: 0.598  loss_rpn_cls: 0.02229  loss_rpn_loc: 0.02744  time: 0.1743  data_time: 0.0030  lr: 0.00029571  max_mem: 1609M
[10/25 13:48:32 d2.utils.events]:  eta: 0:01:13  iter: 79  total_loss: 1.566  loss_cls: 0.4208  loss_box_reg: 0.6982  loss_mask: 0.5083  loss_rpn_cls: 0.0132  loss_rpn_loc: 0.02492  time: 0.1743  data_time: 0.0031  lr: 0.0003956  max_mem: 1609M
[10/25 13:48:36 d2.

...

[10/25 13:51:18 d2.engine.train_loop]: Starting training from iteration 0
[10/25 13:51:22 d2.utils.events]:  eta: 0:02:19  iter: 19  total_loss: 3.16  loss_cls: 1.832  loss_box_reg: 0.6864  loss_mask: 0.6932  loss_rpn_cls: 0.08462  loss_rpn_loc: 0.03259  time: 0.1798  data_time: 0.0141  lr: 9.5905e-05  max_mem: 1609M
[10/25 13:51:25 d2.utils.events]:  eta: 0:02:16  iter: 39  total_loss: 2.088  loss_cls: 0.6967  loss_box_reg: 0.7003  loss_mask: 0.666  loss_rpn_cls: 0.03153  loss_rpn_loc: 0.01657  time: 0.1795  data_time: 0.0034  lr: 0.00019581  max_mem: 1609M
[10/25 13:51:29 d2.utils.events]:  eta: 0:02:11  iter: 59  total_loss: 1.864  loss_cls: 0.5098  loss_box_reg: 0.6527  loss_mask: 0.6002  loss_rpn_cls: 0.01611  loss_rpn_loc: 0.02256  time: 0.1783  data_time: 0.0032  lr: 0.00029571  max_mem: 1609M
[10/25 13:51:33 d2.utils.events]:  eta: 0:02:08  iter: 79  total_loss: 1.602  loss_cls: 0.3761  loss_box_reg: 0.6564  loss_mask: 0.4898  loss_rpn_cls: 0.01283  loss_rpn_loc: 0.03086  time:

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[10/25 13:55:08 d2.engine.train_loop]: Starting training from iteration 0
[10/25 13:55:12 d2.utils.events]:  eta: 0:01:26  iter: 19  total_loss: 3.352  loss_cls: 1.828  loss_box_reg: 0.6923  loss_mask: 0.6918  loss_rpn_cls: 0.1331  loss_rpn_loc: 0.03657  time: 0.1805  data_time: 0.0116  lr: 9.5905e-06  max_mem: 1609M
[10/25 13:55:15 d2.utils.events]:  eta: 0:01:22  iter: 39  total_loss: 3.068  loss_cls: 1.553  loss_box_reg: 0.6382  loss_mask: 0.6879  loss_rpn_cls: 0.103  loss_rpn_loc: 0.02618  time: 0.1800  data_time: 0.0032  lr: 1.958e-05  max_mem: 1609M
[10/25 13:55:19 d2.utils.events]:  eta: 0:01:18  iter: 59  total_loss: 2.569  loss_cls: 1.072  loss_box_reg: 0.6742  loss_mask: 0.6771  loss_rpn_cls: 0.07397  loss_rpn_loc: 0.03083  time: 0.1796  data_time: 0.0032  lr: 2.9571e-05  max_mem: 1609M
[10/25 13:55:22 d2.utils.events]:  eta: 0:01:15  iter: 79  total_loss: 2.283  loss_cls: 0.806  loss_box_reg: 0.7487  loss_mask: 0.6641  loss_rpn_cls: 0.05108  loss_rpn_loc: 0.02313  time: 0.17

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[10/25 13:58:31 d2.engine.train_loop]: Starting training from iteration 0
[10/25 13:58:34 d2.utils.events]:  eta: 0:02:18  iter: 19  total_loss: 3.624  loss_cls: 2.046  loss_box_reg: 0.6844  loss_mask: 0.6922  loss_rpn_cls: 0.08502  loss_rpn_loc: 0.03555  time: 0.1772  data_time: 0.0121  lr: 9.5905e-06  max_mem: 1609M
[10/25 13:58:38 d2.utils.events]:  eta: 0:02:14  iter: 39  total_loss: 3.259  loss_cls: 1.735  loss_box_reg: 0.633  loss_mask: 0.6885  loss_rpn_cls: 0.07373  loss_rpn_loc: 0.02492  time: 0.1784  data_time: 0.0031  lr: 1.958e-05  max_mem: 1609M
[10/25 13:58:42 d2.utils.events]:  eta: 0:02:11  iter: 59  total_loss: 2.657  loss_cls: 1.18  loss_box_reg: 0.6986  loss_mask: 0.6793  loss_rpn_cls: 0.0587  loss_rpn_loc: 0.02386  time: 0.1783  data_time: 0.0031  lr: 2.9571e-05  max_mem: 1609M
[10/25 13:58:45 d2.utils.events]:  eta: 0:02:08  iter: 79  total_loss: 2.367  loss_cls: 0.798  loss_box_reg: 0.7137  loss_mask: 0.6641  loss_rpn_cls: 0.06856  loss_rpn_loc: 0.02579  time: 0.17

In [ ]:
# _runner([1,2,3,4], 5, [2,4,6,8])

dataset_name HDM5R2N
SOLVER PARAMS (500, 200, 0.001)
classes Metadata(evaluator_type='coco', image_root='/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/default/rgb', json_file='/checkpoint/apratik/jobs/active_vision/pipeline/apartment_0/straightline/no_noise/20-10-2021/23:04:37/1/default/pred_label_gt5p2/coco_train.json', name='HDM5R2N_train', thing_classes=['chair', 'cushion', 'door', 'indoor-plant', 'sofa', 'table'])
recreating output_aug/500/HDM5R2N0
output_aug/500/HDM5R2N0
[10/21 11:10:19 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_o

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[10/21 11:10:19 d2.engine.train_loop]: Starting training from iteration 0
[10/21 11:10:23 d2.utils.events]:  eta: 0:01:29  iter: 19  total_loss: 3.083  loss_cls: 1.651  loss_box_reg: 0.6183  loss_mask: 0.6927  loss_rpn_cls: 0.08876  loss_rpn_loc: 0.02216  time: 0.1881  data_time: 0.0173  lr: 9.5905e-05  max_mem: 3097M
[10/21 11:10:27 d2.utils.events]:  eta: 0:01:26  iter: 39  total_loss: 2.136  loss_cls: 0.6881  loss_box_reg: 0.7142  loss_mask: 0.6604  loss_rpn_cls: 0.06532  loss_rpn_loc: 0.03367  time: 0.1918  data_time: 0.0042  lr: 0.0001958  max_mem: 3097M
[10/21 11:10:31 d2.utils.events]:  eta: 0:01:23  iter: 59  total_loss: 1.923  loss_cls: 0.5116  loss_box_reg: 0.7  loss_mask: 0.6121  loss_rpn_cls: 0.02695  loss_rpn_loc: 0.0408  time: 0.1918  data_time: 0.0040  lr: 0.00029571  max_mem: 3097M
[10/21 11:10:35 d2.utils.events]:  eta: 0:01:19  iter: 79  total_loss: 1.674  loss_cls: 0.3987  loss_box_reg: 0.6406  loss_mask: 0.5093  loss_rpn_cls: 0.02592  loss_rpn_loc: 0.03311  time: 0.

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[10/21 11:13:34 d2.engine.train_loop]: Starting training from iteration 0
[10/21 11:13:38 d2.utils.events]:  eta: 0:02:31  iter: 19  total_loss: 2.989  loss_cls: 1.585  loss_box_reg: 0.589  loss_mask: 0.6875  loss_rpn_cls: 0.1013  loss_rpn_loc: 0.027  time: 0.1953  data_time: 0.0173  lr: 9.5905e-05  max_mem: 3097M
[10/21 11:13:42 d2.utils.events]:  eta: 0:02:25  iter: 39  total_loss: 2.104  loss_cls: 0.6667  loss_box_reg: 0.6571  loss_mask: 0.6581  loss_rpn_cls: 0.03702  loss_rpn_loc: 0.0232  time: 0.1926  data_time: 0.0039  lr: 0.00019581  max_mem: 3097M
[10/21 11:13:46 d2.utils.events]:  eta: 0:02:21  iter: 59  total_loss: 1.832  loss_cls: 0.481  loss_box_reg: 0.5982  loss_mask: 0.6075  loss_rpn_cls: 0.02845  loss_rpn_loc: 0.02947  time: 0.1921  data_time: 0.0042  lr: 0.00029571  max_mem: 3097M
[10/21 11:13:50 d2.utils.events]:  eta: 0:02:18  iter: 79  total_loss: 1.518  loss_cls: 0.3668  loss_box_reg: 0.6195  loss_mask: 0.5203  loss_rpn_cls: 0.01396  loss_rpn_loc: 0.02375  time: 0.1

...

[10/21 13:29:07 d2.engine.train_loop]: Starting training from iteration 0
[10/21 13:29:11 d2.utils.events]:  eta: 0:02:36  iter: 19  total_loss: 3.796  loss_cls: 2.011  loss_box_reg: 0.9399  loss_mask: 0.6916  loss_rpn_cls: 0.1082  loss_rpn_loc: 0.03906  time: 0.2012  data_time: 0.0166  lr: 9.5905e-06  max_mem: 3097M
[10/21 13:29:15 d2.utils.events]:  eta: 0:02:31  iter: 39  total_loss: 3.583  loss_cls: 1.813  loss_box_reg: 0.9063  loss_mask: 0.6879  loss_rpn_cls: 0.08653  loss_rpn_loc: 0.04274  time: 0.1999  data_time: 0.0044  lr: 1.958e-05  max_mem: 3097M
[10/21 13:29:19 d2.utils.events]:  eta: 0:02:26  iter: 59  total_loss: 3.097  loss_cls: 1.476  loss_box_reg: 0.9023  loss_mask: 0.6787  loss_rpn_cls: 0.05327  loss_rpn_loc: 0.0306  time: 0.1999  data_time: 0.0043  lr: 2.9571e-05  max_mem: 3097M
[10/21 13:29:23 d2.utils.events]:  eta: 0:02:23  iter: 79  total_loss: 2.755  loss_cls: 1.075  loss_box_reg: 0.9311  loss_mask: 0.6685  loss_rpn_cls: 0.0531  loss_rpn_loc: 0.03511  time: 0.19

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[10/21 13:33:21 d2.engine.train_loop]: Starting training from iteration 0
[10/21 13:33:26 d2.utils.events]:  eta: 0:01:36  iter: 19  total_loss: 3.514  loss_cls: 1.865  loss_box_reg: 0.9089  loss_mask: 0.692  loss_rpn_cls: 0.07625  loss_rpn_loc: 0.03843  time: 0.2019  data_time: 0.0177  lr: 9.5905e-05  max_mem: 3097M
[10/21 13:33:30 d2.utils.events]:  eta: 0:01:31  iter: 39  total_loss: 2.529  loss_cls: 0.8777  loss_box_reg: 0.885  loss_mask: 0.6609  loss_rpn_cls: 0.03514  loss_rpn_loc: 0.02837  time: 0.1999  data_time: 0.0043  lr: 0.0001958  max_mem: 3097M
[10/21 13:33:34 d2.utils.events]:  eta: 0:01:27  iter: 59  total_loss: 2.223  loss_cls: 0.7025  loss_box_reg: 0.8618  loss_mask: 0.5867  loss_rpn_cls: 0.02452  loss_rpn_loc: 0.03474  time: 0.1995  data_time: 0.0042  lr: 0.00029571  max_mem: 3097M
[10/21 13:33:38 d2.utils.events]:  eta: 0:01:23  iter: 79  total_loss: 1.985  loss_cls: 0.5878  loss_box_reg: 0.8378  loss_mask: 0.5023  loss_rpn_cls: 0.02058  loss_rpn_loc: 0.0294  time: 0

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[10/21 13:36:39 d2.engine.train_loop]: Starting training from iteration 0
[10/21 13:36:43 d2.utils.events]:  eta: 0:02:35  iter: 19  total_loss: 3.554  loss_cls: 1.849  loss_box_reg: 0.8877  loss_mask: 0.6906  loss_rpn_cls: 0.1086  loss_rpn_loc: 0.0496  time: 0.2023  data_time: 0.0176  lr: 9.5905e-05  max_mem: 3097M
[10/21 13:36:47 d2.utils.events]:  eta: 0:02:30  iter: 39  total_loss: 2.585  loss_cls: 0.9246  loss_box_reg: 0.8657  loss_mask: 0.6609  loss_rpn_cls: 0.03959  loss_rpn_loc: 0.03201  time: 0.1999  data_time: 0.0044  lr: 0.00019581  max_mem: 3097M
[10/21 13:36:51 d2.utils.events]:  eta: 0:02:26  iter: 59  total_loss: 2.196  loss_cls: 0.723  loss_box_reg: 0.8254  loss_mask: 0.585  loss_rpn_cls: 0.02602  loss_rpn_loc: 0.02868  time: 0.1994  data_time: 0.0041  lr: 0.00029571  max_mem: 3097M
[10/21 13:36:55 d2.utils.events]:  eta: 0:02:22  iter: 79  total_loss: 1.955  loss_cls: 0.602  loss_box_reg: 0.8207  loss_mask: 0.4831  loss_rpn_cls: 0.01942  loss_rpn_loc: 0.02527  time: 0.